In [2]:
import pandas as pd

In [5]:
tablero_vacantes= pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Actualizacion de informacion\Vacantes 2025.xlsx")
tablero_aps = pd.read_excel(r"C:\Users\centro.control\Downloads\TableroAps-2025-07-07.xlsx", skiprows=[1])
id_sitios = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\SitioActivosPC.xlsx")

In [30]:
id_sitios = id_sitios.rename(columns={
    'key_service': 'id',
    'business_name': 'Sitio'}
)

In [ ]:
def tabla_turno(turno_str):
    # Determinar índice y nombre de columna
    if turno_str == 'Matutino':
        i = 6
        aux = 'MAT'
        turno= 'Matutino'

    elif turno_str == 'Vespertino':
        i = 9
        aux = 'Vesp'
        turno= 'Vespertino'

    elif turno_str == 'Nocturno':
        i = 12
        aux = 'Noc'
        turno= 'Nocturno'

    elif turno_str == 'Mixto':
        i = 15
        aux = 'Mix'
        turno= 'Mixto'

    else:
        raise ValueError("Turno no válido. Usa 'Matutino', 'Vespertino', 'Nocturno' o 'Mixto'.")

    # Selección y renombramiento de columnas
    columnas = ['FECHA', 'SITIO', aux, f'Unnamed: {i}', f'Unnamed: {i+1}']
    turno_df = tablero_aps[columnas].copy()

    turno_df = turno_df.rename(columns={
        'FECHA': 'Fecha del reporte',
        'SITIO': 'Sitio',
        aux: 'Plantilla',
        f'Unnamed: {i}': 'Activos',
        f'Unnamed: {i+1}': 'Vacante'
    })

    turno_df['Turno'] = turno
    turno_df['Tipo'] = 'Natural'
    turno_df['Ejecutivo'] = ''



    return turno_df


def cruzar_id(turno):
    turno = turno.merge(id_sitios ,on='Sitio', how='left').drop('site_manager', axis=1).fillna(0)
    turno['id'] = turno['id'].astype(int) 
    turno = turno[['id','Fecha del reporte', 'Sitio', 'Plantilla', 'Activos', 'Vacante','Turno', 'Tipo', 'Ejecutivo']]
    return turno


In [32]:
Matutino = tabla_turno('Matutino')
Vespertino = tabla_turno('Vespertino')
Nocturno = tabla_turno('Nocturno')
Mixto = tabla_turno('Mixto')

In [33]:
Matutino = cruzar_id(Matutino)
Vespertino = cruzar_id(Vespertino)
Nocturno = cruzar_id(Nocturno)
Mixto = cruzar_id(Mixto)

In [34]:
tablero_actualizado = pd.concat([tablero_vacantes, Matutino, Vespertino, Nocturno, Mixto], ignore_index=True)

In [35]:
tablero_actualizado['Fecha del reporte'] = pd.to_datetime(tablero_actualizado['Fecha del reporte'])
tablero_actualizado['Fecha del reporte'] = tablero_actualizado['Fecha del reporte'].dt.date
tablero_actualizado = tablero_actualizado.drop_duplicates()

In [36]:
tablero_actualizado.to_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Actualizacion de informacion\Vacantes 2025.xlsx", index=False)